In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [8]:
map = gpd.read_file("Sidewalk_width_crossings_smaller.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossings')

In [9]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
edges = G.edges

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length
/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length
/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [10]:
print(nodes)

[(122245.37633330293, 486126.8581684635), (122254.86602688645, 486129.80052856216), (122264.35426393585, 486132.74411788705), (122273.84081337851, 486135.69101398275), (122283.19823912054, 486138.9678026403), (122284.1816391946, 486139.19731384714), (122287.28805009159, 486137.97386694513), (122291.40829399273, 486140.643590483), (122289.53424561612, 486128.31132216856), (122297.7854183222, 486139.47700177913), (122301.24255780602, 486130.1701925009), (122304.47424118801, 486120.76208520425), (122256.54763350038, 486170.91398380254), (122247.04588395767, 486167.91526857053), (122237.50744934028, 486164.9669819529), (122227.969152229, 486162.0189639581), (122218.43104837791, 486159.0709819753), (122289.24584929895, 486173.25169782597), (122282.43320919861, 486172.4559260068), (122296.3898333376, 486151.2069021054), (122293.35529070727, 486160.70100324217), (122290.30564961619, 486170.1963395323), (122290.12705889999, 486148.4950394649), (122234.28219718198, 486227.4182083386), (122243.0

In [11]:
#Pick random ones or pick manually that make sense - to experiment
S = (122245.37633330293, 486126.8581684635)
T = (122320.31466476223, 486327.5294561802)

In [107]:
#TODO: How does PVI give me a path to compare against my algo?
def pareto_dominates(a, b):
    """Check if the vector in a Pareto dominates vector b.

    Args:
        a (ndarray): A numpy array.
        b (ndarray): A numpy array.

    Returns:
        bool: Whether vector a dominates vector b.
    """
    a = np.array(a)
    b = np.array(b)
    return np.all(a >= b) and np.any(a > b)

def p_prune(candidates):
    """Create a Pareto coverage set from a set of candidate points.

    References:
        .. [1] Roijers, D. M., & Whiteson, S. (2017). Multi-objective decision making. 34, 129–129.
            https://doi.org/10.2200/S00765ED1V01Y201704AIM034

    Args:
        candidates (Set[Tuple]): A set of vectors.

    Returns:
        Set[Tuple]: A Pareto coverage set.
    """
    pcs = []

    while candidates:
        vector = candidates.pop()

        for alternative in candidates:
            if pareto_dominates(alternative, vector):
                vector = alternative

        to_remove = vector
        for alternative in candidates:
            if pareto_dominates(vector, alternative):
                to_remove.add(alternative)

        # candidates -= to_remove
        # candidates = np.delete(candidates, np.where(np.all(candidates == to_remove)))
        indices = [i for i, x in enumerate(candidates) if np.all(x == to_remove)]
        for index in sorted(indices, reverse=True):
            del candidates[index]

        pcs.append(vector)
    return pcs


v_n = {}  # Value vector
objectives = ('length', 'crossing')

for n in G:
    v_n[n] = np.inf  # Initialisation of nodes

    if n == T:
        v_n[n] = 0

    for neighbor in G.neighbors(n):
        edge = G[n][neighbor]
        edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties
        v_n[neighbor] = np.array([edge_list[0]['length'], edge_list[0]['crossing']])
        for i in objectives:
            cost = edge_list[0][i]

        result = cost + v_n[neighbor]
        pruned = p_prune([result])

    v_n[n] = pruned

print(v_n)

#Without prune: 177 v_n
#With prune: 152 v_n

{(122245.37633330293, 486126.8581684635): array([9.99, 0.  ]), (122254.86602688645, 486129.80052856216): array([9.99, 0.  ]), (122264.35426393585, 486132.74411788705): array([9.99, 0.  ]), (122273.84081337851, 486135.69101398275): array([9.99, 0.  ]), (122283.19823912054, 486138.9678026403): array([1.01, 0.  ]), (122284.1816391946, 486139.19731384714): array([11.04,  1.  ]), (122287.28805009159, 486137.97386694513): array([9.99, 0.  ]), (122290.12705889999, 486148.4950394649): [array([12.04,  2.  ])], (122291.40829399273, 486140.643590483): array([6.57, 0.  ]), (122289.53424561612, 486128.31132216856): [array([9.99, 0.  ])], (122297.7854183222, 486139.47700177913): array([40.73,  1.  ]), (122301.24255780602, 486130.1701925009): array([9.99, 0.  ]), (122333.58719518047, 486150.8910123211): array([9.99, 0.  ]), (122332.49987764945, 486160.7773385103): array([2.93, 0.  ]), (122304.47424118801, 486120.76208520425): [array([9.99, 0.  ])], (122256.54763350038, 486170.91398380254): array([19.